<a href="https://colab.research.google.com/github/apchavezr/-Analisis_Grandes_Volumenes_Datos/blob/main/Comparacion_MapReduce_vs_Spark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Comparación de latencia entre MapReduce y Spark

Este cuaderno permite comparar la ejecución de un mismo algoritmo (conteo de palabras) usando una simulación del modelo MapReduce en Python y PySpark. Se observarán diferencias en latencia y modelo de procesamiento.

In [1]:
# Instalación de PySpark en el entorno (solo necesario una vez)
!pip install pyspark

In [2]:
# Se crea una sesión de Spark y el contexto de SparkContext
from pyspark.sql import SparkSession
import time

# Se crea una sesión de Spark y el contexto de SparkContext
spark = SparkSession.builder.appName("ComparacionMapReduceSpark").getOrCreate()
sc = spark.sparkContext

## Paso 1: Simulación del algoritmo MapReduce en Python

In [3]:
from collections import defaultdict

# Texto de ejemplo que será procesado con ambos modelos
texto = "big data requiere procesamiento distribuido y spark acelera procesos big data big spark"

# Se inicia el conteo de tiempo para medir la latencia del modelo
inicio_mapreduce = time.time()

# Map
# Fase Map: convierte cada palabra en una tupla (palabra, 1)
map_output = [(palabra, 1) for palabra in texto.split()]

# Reduce
# Fase Reduce: acumula las ocurrencias de cada palabra
reduce_output = defaultdict(int)
for palabra, cuenta in map_output:
# Fase Reduce: acumula las ocurrencias de cada palabra
    reduce_output[palabra] += cuenta

fin_mapreduce = time.time()

# Se imprime el resultado y el tiempo de ejecución
print("Resultado MapReduce (Python):")
# Fase Reduce: acumula las ocurrencias de cada palabra
for palabra, cuenta in reduce_output.items():
# Se imprime el resultado y el tiempo de ejecución
    print(f"{palabra}: {cuenta}")

# Se inicia el conteo de tiempo para medir la latencia del modelo
print(f"Tiempo de ejecución MapReduce simulado: {fin_mapreduce - inicio_mapreduce:.6f} segundos")

Resultado MapReduce (Python):
big: 3
data: 2
requiere: 1
procesamiento: 1
distribuido: 1
y: 1
spark: 2
acelera: 1
procesos: 1
Tiempo de ejecución MapReduce simulado: 0.000182 segundos


## Paso 2: Implementación del mismo algoritmo con PySpark (RDD)

In [4]:
# Se inicia el conteo de tiempo para medir la latencia del modelo
inicio_spark = time.time()

# Se paraleliza el texto dividiéndolo en palabras con Spark
palabras = sc.parallelize(texto.split())
# Fase Map y Reduce usando PySpark con RDDs
conteo = palabras.map(lambda x: (x, 1)).reduceByKey(lambda a, b: a + b)
# Se recopilan los resultados distribuidos al driver
resultado = conteo.collect()

fin_spark = time.time()

# Se imprime el resultado y el tiempo de ejecución
print("Resultado PySpark RDD:")
for palabra, cuenta in resultado:
# Se imprime el resultado y el tiempo de ejecución
    print(f"{palabra}: {cuenta}")

# Se inicia el conteo de tiempo para medir la latencia del modelo
print(f"Tiempo de ejecución PySpark: {fin_spark - inicio_spark:.6f} segundos")

Resultado PySpark RDD:
big: 3
requiere: 1
procesamiento: 1
distribuido: 1
data: 2
y: 1
spark: 2
acelera: 1
procesos: 1
Tiempo de ejecución PySpark: 4.067306 segundos


## Paso 3: Comparación de resultados y explicación

- **MapReduce en Python** ejecuta el procesamiento de forma secuencial, con uso intensivo de lectura y escritura en memoria local.
- **PySpark RDD** permite ejecutar el mismo algoritmo en paralelo, distribuyendo las operaciones entre los nodos virtuales del motor Spark.
- Aunque en este entorno la diferencia de tiempo puede no ser significativa, en volúmenes de datos mayores Spark ofrece ventajas claras en latencia al usar procesamiento en memoria.

### Reflexión:
- ¿Qué cambios observa si repite el ejercicio con un texto 100 veces más largo?
- ¿Qué ventajas tiene el modelo in-memory de Spark frente al modelo secuencial?
